In [ ]:
class DO2_profile:   
    '''
    class Nut_depth(resultpath,savepath,meshpath,ncfileDSi,ncfileDO2,ncfileDFe,first_year,last_year,
                 savefig=False,regional=True,runname)
                 
    c.f. Danilov et al. (2017):
    "in the vertical direction, the horizontal velocities and scalars are located at mid-levels" 
                 
    if regional = True, profiles will plotted for each main basins + Global Ocean. 
    Otherwise, just the Global Ocean.
    '''
    def __init__(self,resultpath,savepath,mesh,ncfileDO2,
                 first_year,last_year,savefig=False, regional=True,runname='fesom'):

        self.runname = runname
        self.resultpath = resultpath
        self.savepath = savepath
        self.mesh = mesh
        self.ncfileDO2 = ncfileDO2
        self.fyear = first_year
        self.lyear = last_year
        self.savefig = savefig
        self.regional = regional
        
        import matplotlib.pyplot as plt
        import numpy as np
        from netCDF4 import Dataset
        from scipy.interpolate import griddata
        import skill_metrics as sm
        import cartopy.crs as ccrs
        import pyfesom2 as pf
        from load_interp_WOA import WOAdata
        from load_interp_PISCES import PISCESdata
        
        years = np.arange(first_year, last_year+1,1)
        meshdiag = pf.get_meshdiag(mesh)
        runid      =  self.runname
        
        unitsDO2 = 'DO2 [mmol m$^{-3}$]'
        
        
        # load data -------------------------------------------------------------------------------------
        DO2fesom = pf.get_data(resultpath, "O2", years, mesh,
                               how="mean", compute=True, runid=runid, silent=True)

        DO2woa_input = WOAdata(runid,resultpath,mesh,ncfileDO2,'oxygen_mmol', get_overview=False)

        DO2woa = DO2woa_input.woa_int
        DO2woa[DO2fesom == 0] = 0
        
        # Load and derive profiles

        nod_area = np.ma.masked_equal(meshdiag.nod_area.values, 0)
        mask = pf.get_mask(mesh, "Global Ocean")

        DO2fesom_by_area = ((np.ma.masked_equal(DO2fesom[mask,:],0) * nod_area[:-1,:].T[mask]).mean(axis=0))
        DO2woa_by_area = ((np.ma.masked_equal(DO2woa[mask,:],0) * nod_area[:-1,:].T[mask]).mean(axis=0))

        DO2fesom_weighted_Global = DO2fesom_by_area/nod_area[:-1,:].T[mask].mean(axis=0)
        DO2woa_weighted_Global = DO2woa_by_area/nod_area[:-1,:].T[mask].mean(axis=0)
        
        if regional:
            mask = pf.get_mask(mesh, "Atlantic_Basin")

            DO2fesom_by_area = ((np.ma.masked_equal(DO2fesom[mask,:],0) * nod_area[:-1,:].T[mask]).mean(axis=0))
            DO2woa_by_area = ((np.ma.masked_equal(DO2woa[mask,:],0) * nod_area[:-1,:].T[mask]).mean(axis=0))

            DO2fesom_weighted_Atlantic = DO2fesom_by_area/nod_area[:-1,:].T[mask].mean(axis=0)
            DO2woa_weighted_Atlantic = DO2woa_by_area/nod_area[:-1,:].T[mask].mean(axis=0)

            mask = pf.get_mask(mesh, "Pacific_Basin")

            DO2fesom_by_area = ((np.ma.masked_equal(DO2fesom[mask,:],0) * nod_area[:-1,:].T[mask]).mean(axis=0))
            DO2woa_by_area = ((np.ma.masked_equal(DO2woa[mask,:],0) * nod_area[:-1,:].T[mask]).mean(axis=0))

            DO2fesom_weighted_Pacific = DO2fesom_by_area/nod_area[:-1,:].T[mask].mean(axis=0)
            DO2woa_weighted_Pacific = DO2woa_by_area/nod_area[:-1,:].T[mask].mean(axis=0)

            mask = pf.get_mask(mesh, "Indian_Basin")

            DO2fesom_by_area = ((np.ma.masked_equal(DO2fesom[mask,:],0) * nod_area[:-1,:].T[mask]).mean(axis=0))
            DO2woa_by_area = ((np.ma.masked_equal(DO2woa[mask,:],0) * nod_area[:-1,:].T[mask]).mean(axis=0))

            DO2fesom_weighted_Indian = DO2fesom_by_area/nod_area[:-1,:].T[mask].mean(axis=0)
            DO2woa_weighted_Indian = DO2woa_by_area/nod_area[:-1,:].T[mask].mean(axis=0)
            
            mask = pf.get_mask(mesh, "Arctic_Basin")

            DO2fesom_by_area = ((np.ma.masked_equal(DO2fesom[mask,:],0) * nod_area[:-1,:].T[mask]).mean(axis=0))
            DO2woa_by_area = ((np.ma.masked_equal(DO2woa[mask,:],0) * nod_area[:-1,:].T[mask]).mean(axis=0))

            DO2fesom_weighted_Arctic = DO2fesom_by_area/nod_area[:-1,:].T[mask].mean(axis=0)
            DO2woa_weighted_Arctic = DO2woa_by_area/nod_area[:-1,:].T[mask].mean(axis=0)
            
            mask = pf.get_mask(mesh, "Southern_Ocean_Basin")

            DO2fesom_by_area = ((np.ma.masked_equal(DO2fesom[mask,:],0) * nod_area[:-1,:].T[mask]).mean(axis=0))
            DO2woa_by_area = ((np.ma.masked_equal(DO2woa[mask,:],0) * nod_area[:-1,:].T[mask]).mean(axis=0))

            DO2fesom_weighted_Southern = DO2fesom_by_area/nod_area[:-1,:].T[mask].mean(axis=0)
            DO2woa_weighted_Southern = DO2woa_by_area/nod_area[:-1,:].T[mask].mean(axis=0)
            
            # plotting
            
            fig, axs = plt.subplots(1,6, figsize=(14, 4), facecolor='w', edgecolor='k', constrained_layout=True, sharey=True)

            axs[0].plot(DO2fesom_weighted_Global, mesh.zlev[:-1]/1000,label = 'FESOM', color = 'k', lw=3)
            axs[0].plot(DO2woa_weighted_Global, mesh.zlev[:-1]/1000,label = 'WOA', color = 'k', lw=3, linestyle = '--')
            axs[0].set_ylabel('Depth [km]',fontsize=14)
            axs[0].set_xlabel(unitsDO2,fontsize=14)
            axs[0].set_title('Global Ocean',size=16, weight='bold')
            axs[0].tick_params(labelsize=14)
            axs[0].grid()
            axs[0].legend(loc='best', borderaxespad=0., fontsize=14)
            
            axs[1].plot(DO2fesom_weighted_Pacific, mesh.zlev[:-1]/1000,label = 'FESOM', color = 'C0', lw=3)
            axs[1].plot(DO2woa_weighted_Pacific, mesh.zlev[:-1]/1000,label = 'WOA', color = 'C0', lw=3, linestyle = '--')
            axs[1].set_xlabel(unitsDO2,fontsize=14)
            axs[1].set_title('Pacific Ocean',size=16, weight='bold')
            axs[1].tick_params(labelsize=14)
            axs[1].grid()
            
            axs[2].plot(DO2fesom_weighted_Atlantic, mesh.zlev[:-1]/1000,label = 'FESOM', color = 'C1', lw=3)
            axs[2].plot(DO2woa_weighted_Atlantic, mesh.zlev[:-1]/1000,label = 'WOA', color = 'C1', lw=3, linestyle = '--')
            axs[2].set_xlabel(unitsDO2,fontsize=14)
            axs[2].set_title('Atlantic Ocean',size=16, weight='bold')
            axs[2].tick_params(labelsize=14)
            axs[2].grid()
            
            axs[3].plot(DO2fesom_weighted_Arctic, mesh.zlev[:-1]/1000,label = 'FESOM', color = 'C2', lw=3)
            axs[3].plot(DO2woa_weighted_Arctic, mesh.zlev[:-1]/1000,label = 'WOA', color = 'C2', lw=3, linestyle = '--')
            axs[3].set_xlabel(unitsDO2,fontsize=14)
            axs[3].set_title('Arctic Ocean',size=16, weight='bold')
            axs[3].tick_params(labelsize=14)
            axs[3].grid()
            
            axs[4].plot(DO2fesom_weighted_Southern, mesh.zlev[:-1]/1000,label = 'FESOM', color = 'C3', lw=3)
            axs[4].plot(DO2woa_weighted_Southern, mesh.zlev[:-1]/1000,label = 'WOA', color = 'C3', lw=3, linestyle = '--')
            axs[4].set_xlabel(unitsDO2,fontsize=14)
            axs[4].set_title('Southern Ocean',size=16, weight='bold')
            axs[4].tick_params(labelsize=14)
            axs[4].grid()
            
            axs[5].plot(DO2fesom_weighted_Indian, mesh.zlev[:-1]/1000,label = 'FESOM', color = 'C4', lw=3)
            axs[5].plot(DO2woa_weighted_Indian, mesh.zlev[:-1]/1000,label = 'WOA', color = 'C4', lw=3, linestyle = '--')
            axs[5].set_xlabel(unitsDO2,fontsize=14)
            axs[5].set_title('Indian Ocean',size=16, weight='bold')
            axs[5].tick_params(labelsize=14)
            axs[5].grid()

        else:
            
            fig, axs = plt.subplots(1,1, figsize=(10, 5), facecolor='w', edgecolor='k', constrained_layout=True, sharey=True)

            axs[0].plot(DO2fesom_weighted_Global, mesh.zlev[:-1]/1000,label = 'FESOM', color = 'k', lw=3)
            axs[0].plot(DO2woa_weighted_Global, mesh.zlev[:-1]/1000,label = 'WOA', color = 'k', lw=3, linestyle = '--')
            axs[0].set_ylabel('Depth [km]',fontsize=14)
            axs[0].set_xlabel(unitsDO2,fontsize=14)
            axs[1].set_title('Global Ocean',size=16, weight='bold')
            axs[0].tick_params(labelsize=14)
            axs[0].grid()
            axs[0].legend(loc='best', borderaxespad=0., fontsize=14)
            
        if(self.savefig == True):
                plt.savefig(self.savepath+self.runname+'_DO2_profiles_'+str(self.fyear)+'to'+str(self.lyear)+'.png', 
                            dpi = 300, bbox_inches='tight')
                plt.savefig(self.savepath+self.runname+'_DO2_profiles_'+str(self.fyear)+'to'+str(self.lyear)+'.pdf', 
                            bbox_inches='tight')
        plt.show(block=False)